# Что нужно сделать 
1. Удалить structure_
2. Геометрические признаки ( сферичность - не получается нормально считать)
3. Total_magnetization по формуле (сумма 2-3)/const (из примера)
3. Pymatgen 
[
1)  Массовое соотношение атомов
2)  Number of magnetic ions per unit cell
3) get_exchange_group_info
4) number_of_unique_magnetic_sites
5) number_of_magnetic_sites
]

In [289]:
# All library connection
import pandas as pd
import numpy as np
import re #регулярки
from math import gcd, isclose
import sympy as sp #библиотека символьных вычеслений, мат.задачи
import matplotlib.pyplot as plt
import seaborn as sns

In [290]:
from pymatgen.core.structure import Composition

In [291]:
df = pd.read_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\Bias_data_1.csv')

In [292]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         533 non-null    int64  
 1   c1                 533 non-null    object 
 2   c2                 322 non-null    object 
 3   c3                 9 non-null      object 
 4   chemical_formula   533 non-null    object 
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   id                 533 non-null    int64  
 8   space_group_shell  533 non-null    int64  
 9   structure          533 non-null    object 
 10  structure_         533 non-null    object 
 11  formula_combined   533 non-null    object 
 12  coer_oe            533 non-null    float64
 13  exc_bias_oe        533 non-null    float64
 14  fc_field_t         533 non-null    float64
 15  h_range_max_koe    533 non-null    float64
 16  length_d_nm        533 non

# Исправим стотлбец formula_combined

Проблема была в том, что формулы, что должны были иметь вид Li2 Cr2 O4 имели вид Li2 Cr1 O3 Cr1 O1

In [293]:
print(df['formula_combined'].unique()) # Пример, что нас не устраивает

['Zn1 O1 Fe2 Co1 O4' 'Co1 O1 Fe2 Co1 O4' 'Mn1 Zn1 Fe4 O8' 'La1 Fe1 O3'
 'Ni1 Ni1 O1' 'Mg1 Fe2 O4' 'Fe2 Co1 O4' 'Fe1 Bi1 O3'
 'Fe2 Co1 O4 Fe1 Bi1 O3' 'Co1 Co1 O1' 'Fe1 Rh1' 'Mn1 O2' 'Ag1 Fe3 O4'
 'Fe1 Fe2 O3' 'Fe1 O1 Fe3 O4' 'Ni1 O1' 'Fe3 O4' 'Fe3 O4 Ni1 O1'
 'Ni1 O1 Fe3 O4' 'Fe4 Bi2 O9 Fe1 Bi1 O3' 'Fe1 Bi1 O3 Fe4 Bi2 O9'
 'Ti1 O2 Fe2 O3' 'Fe13 Co2 O20' 'Fe23 Co7 O40' 'Fe7 Co3' 'Fe1 Co1'
 'Fe3 Co7' 'Fe1 Fe3 O4' 'La2 Mn1 Fe1 O6' 'V1 Fe1 O4 Fe3 O4' 'Au1 Fe3 O4'
 'Gd1 Ti1 Fe9 Bi9 O30' 'Sr11 Nd39 Co50 Cu97 O150' 'Fe3 O4 Co1 O1'
 'Fe2 Co1 O4 Co1 O1' 'Cu98 Fe1 O50' 'Cu96 Fe1 O25' 'Fe2 O3'
 'Sr3 La17 Fe20 O60' 'Co1 Co3 O4' 'Mn3 O4' 'Sr33 La67 Mn100 O300'
 'Mn1 O1 Mn3 O4' 'Fe3 O4 Fe2 O3' 'Co3 O4' 'Fe1 Ni99 O100' 'Fe1 Ni49 O50'
 'Fe1 Ni24 O25' 'Fe3 Ni97 O100' 'Cr5 Ag1 O8' 'Cr1 Ag1 O2'
 'Co1 O1 Fe2 Ni1 O4' 'Mn1 Fe2 O4' 'Mn1 Fe2 O4 Fe3 O4' 'La1 Fe10 Bi9 O30'
 'Nd1 Fe1 O3' 'Fe2 Co1 O4 Fe3 O4' 'Dy1 Ti1 Cr1 O5' 'Gd1 Ti1 Cr1 O5'
 'Ni1 O1 Mn2 Co1 O4' 'Ba1 Fe12 O19 Fe3 O4' 'Fe2 Ni1 O4 Co1 O1'
 'Cr2 Co1 

Пример работы с такой ошибкой

1. Создание объекта Composition из вашей формулы
comp = Composition("Zn1 O1 Fe2 Co1 O4") # Может теперь можно склеить их обратно

2. Получение словаря количества атомов каждого элемента
el_amt_dict = comp.get_el_amt_dict()

3. Собираем обратно химическую формулу из словаря
formula = ""
for el, amt in el_amt_dict.items():
    formula += f"{el}{'' if amt == 1 else str(int(amt))}"

print(Composition(formula))

In [294]:
def transform_formula(formula):
    '''
    Zn1 O1 Fe2 Co1 O4 -> {'Zn': 1.0, 'O': 5.0, 'Fe': 2.0, 'Co': 1.0} -> Zn1 O5 Fe2 Co1
    '''
    comp = Composition(formula)
    el_amt_dict = comp.get_el_amt_dict() # Получение словаря с количеством атомов в формуле
    # Обратная сборка
    new_formula = ''
    for el, amt in el_amt_dict.items():
        new_formula += f"{el}{'' if amt == 1 else str(int(amt))}"
    return new_formula

In [295]:
def process_column(df, column_name):
    """
    Функция проверяет каждую ячейку в столбце DataFrame на равенство 0 или NaN.
    Если значение не равно 0 или NaN, оно заменяется на результат применения класса Composition из библиотеки pymatgen.

    Args:
        df (pandas.DataFrame): Исходный DataFrame.
        column_name (str): Имя столбца, который нужно обработать.

    Returns:
        pandas.DataFrame: DataFrame c обработанным столбцом.
    """
    new_column = []
    for value in df[column_name]:
        if pd.notna(value) and value != 0:
            # Если значение не NaN и не равно 0, применяем класс Composition
            comp = Composition(str(value))
            new_value = comp.formula
        else:
            new_value = value
        new_column.append(new_value)

    df[column_name] = new_column
    return df

In [296]:
df['formula_combined'] = df['formula_combined'].apply(transform_formula)
df = process_column(df, 'formula_combined')
print(df['formula_combined'].unique()) # Тепрь у нас есть полный средний материал ядра+оболочки

['Zn1 Fe2 Co1 O5' 'Fe2 Co2 O5' 'Mn1 Zn1 Fe4 O8' 'La1 Fe1 O3' 'Ni2 O1'
 'Mg1 Fe2 O4' 'Fe2 Co1 O4' 'Fe1 Bi1 O3' 'Fe3 Co1 Bi1 O7' 'Co2 O1'
 'Fe1 Rh1' 'Mn1 O2' 'Fe3 Ag1 O4' 'Fe3 O3' 'Fe4 O5' 'Ni1 O1' 'Fe3 O4'
 'Fe3 Ni1 O5' 'Fe5 Bi3 O12' 'Ti1 Fe2 O5' 'Fe13 Co2 O20' 'Fe23 Co7 O40'
 'Fe7 Co3' 'Fe1 Co1' 'Fe3 Co7' 'Fe4 O4' 'La2 Mn1 Fe1 O6' 'V1 Fe4 O8'
 'Fe3 Au1 O4' 'Gd1 Ti1 Fe9 Bi9 O30' 'Sr11 Nd39 Co50 Cu97 O150'
 'Fe3 Co1 O5' 'Fe1 Cu98 O50' 'Fe1 Cu96 O25' 'Fe2 O3' 'Sr3 La17 Fe20 O60'
 'Co4 O4' 'Mn3 O4' 'Sr33 La67 Mn100 O300' 'Mn4 O5' 'Fe5 O7' 'Co3 O4'
 'Fe1 Ni99 O100' 'Fe1 Ni49 O50' 'Fe1 Ni24 O25' 'Fe3 Ni97 O100'
 'Cr5 Ag1 O8' 'Cr1 Ag1 O2' 'Fe2 Co1 Ni1 O5' 'Mn1 Fe2 O4' 'Mn1 Fe5 O8'
 'La1 Fe10 Bi9 O30' 'Nd1 Fe1 O3' 'Fe5 Co1 O8' 'Dy1 Ti1 Cr1 O5'
 'Gd1 Ti1 Cr1 O5' 'Mn2 Co1 Ni1 O5' 'Ba1 Fe15 O23' 'Cr2 Co1 O4' 'Fe1 Ni1'
 'Mn4 O4' 'Mn1 Fe3 O5' 'Fe4 C1' 'Mg1 Fe3 O5' 'Mg1 Fe5 Co1 O9' 'Fe2 Co1 O3'
 'Fe4 Co1 Ni1 O8' 'Fe2 Ni1 O4' 'Mn1 Fe4 O7' 'Fe8 C2 O4' 'Fe5 C2' 'Fe1 Ni3'
 'Fe1 Ni3 Mo1' 'Fe11 C5' 'Fe11 

# Добавление дескрипторов 
## Pymatgen

// TODO: добавление параемтров обязательно 
1, 3 -  можно посчитать для всего материала ядро-оболочка в целом, наверное

1. Массовое соотношение атомов  +
2. Number of magnetic ions per unit cell
3. get_exchange_group_info
4. number_of_unique_magnetic_sites
5. number_of_magnetic_sites

In [297]:
from pymatgen.core.composition import Composition

compound_formula = "Fe2O3"  # пример химической формулы вещества
composition = Composition(compound_formula)
total_num_magnetic_ions = sum(composition.get_el_amt_dict().values())  # суммирование количества ионов для каждого элемента
print("Total number of magnetic ions per unit cell:", total_num_magnetic_ions)

Total number of magnetic ions per unit cell: 5.0


In [298]:
# Функция для расчёта Массового соотношения атомов из пункта 1 
def calculate_mass_ratio(formula): 
    comp = Composition(formula)
    return comp.weight / comp.num_atoms # 

In [299]:
df['mass_ratio'] = df['formula_combined'].apply(lambda x: calculate_mass_ratio(x))
print(df['mass_ratio'])

0      35.114355
1      34.394821
2      33.694446
3      33.694446
4      33.694446
         ...    
528    45.348781
529    45.348781
530    45.348781
531    45.348781
532    45.348781
Name: mass_ratio, Length: 533, dtype: float64


## Геометрические признаки

In [300]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         533 non-null    int64  
 1   c1                 533 non-null    object 
 2   c2                 322 non-null    object 
 3   c3                 9 non-null      object 
 4   chemical_formula   533 non-null    object 
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   id                 533 non-null    int64  
 8   space_group_shell  533 non-null    int64  
 9   structure          533 non-null    object 
 10  structure_         533 non-null    object 
 11  formula_combined   533 non-null    object 
 12  coer_oe            533 non-null    float64
 13  exc_bias_oe        533 non-null    float64
 14  fc_field_t         533 non-null    float64
 15  h_range_max_koe    533 non-null    float64
 16  length_d_nm        533 non

In [301]:
df.rename(columns={'width_nm': 'x', 'length_d_nm': 'y', 'depth_nm': 'z'}, inplace=True)  # Для удобства теперь x, y, z

Помним что все наши формы закодированы:
'''
{
    1: ['sphere', 'quasi-spherical', 'quasi-sphere', 'pseudo-spherical', 'pseudospherically', 'egg', 'flower'], 
    2: ['octahedral', 'octahedron', 'octahedral '],  
    3: ['cubic', 'cube'],
    4: ['hexahedron', 'hexagonal'],
    5: ['tetrahedral', 'triangle'],
    6: ['wire', 'rod'], 
    7: ['octopod']
}
'''

In [302]:
print(df['shape'].unique())

[1. 5. 2. 6. 3. 7. 4.]


In [303]:
# Добавим колонки для наших значений 
df.insert(0, 'area', 0)
df.insert(0, 'volume', 0)
df = df.astype({'volume': float ,'area': float})

In [304]:
# Создадим словари для подсчёта площади и объёма
dict_area = {
    1: lambda x, y=1, z=1: 4 * 3.14159 * x ** 2, 
    2: lambda x, y=1, z=1: (3 ** 2) * 2 * x ** 2, 
    3: lambda x, y=1, z=1: 6 * x **2, 
    4: lambda x, y=1, z=1: 2 * x ** 2 + 4 * x * z/2,   # TODO: #Разделить шестиугольную призму и hexahedron
    5: lambda x, y=1, z=1: (3 ** (1/2)) * x ** 2, 
    6: lambda x, y, z=1: 2 * 3.14 * (x/2) * x +  2 * 3.14 * (x/2) * x ** 2, 
    7: lambda x, y=1, z=1: (2 ** (1/2)) * 2 * x ** 2   
}

dict_volume = {
    1: lambda x, y=1, z=1: (4/3) * 3.14159 * x ** 3, 
    2: lambda x, y=1, z=1: (2/3) * (2 ** 2) * x ** 3, 
    3: lambda x, y=1, z=1: x ** 3, 
    4: lambda x, y=1, z =1: x ** 2 * x/2,  # TODO: разделить формы 
    5: lambda x , y=1, z=1: (((2) ** (1/2)) * x ** 3) / 12,  
    6: lambda x, y, z=1: 3.14159 * (x ** 2) * (y / 2),
    7: lambda x, y=1, z=1: (2/3) * (2 ** (1/2)) * x ** 3
}

In [305]:
# Создаем функцию для вычисления площади
def calculate_area(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)
    
    if shape in dict_area:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_area[shape](x, y) if shape in [3, 6] else dict_area[shape](x, z) if shape == 4 else dict_area[shape](x)
    else:
        return 'error'

# Создаем функцию для вычисления объема
def calculate_volume(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)
    
    if shape in dict_volume:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_volume[shape](x, y, z) if shape in [3, 6] else dict_volume[shape](x, z) if shape == 4 else dict_volume[shape](x)
    else:
        return 'error'

In [306]:
df['area'] = df.apply(calculate_area, axis=1)
df['volume'] = df.apply(calculate_volume, axis=1)

In [307]:
print(df['area'].unique())

[6.88133874e+02 6.15751640e+02 3.63167804e+03 1.52052956e+03
 1.80955584e+03 4.07150064e+03 5.54176476e+03 9.16087644e+03
 1.62860026e+04 1.13097240e+04 2.49415316e+02 4.43405007e+02
 1.03680000e+04 1.96020000e+04 3.93691492e+03 5.22961638e+03
 7.42030992e+03 1.36019537e+04 7.85397500e+03 5.02654400e+05
 2.63760000e+04 5.30928710e+04 1.70553780e+03 3.76098588e+03
 4.52310900e+03 1.36847660e+04 3.40055805e+04 1.26676763e+04
 4.53645596e+03 2.12371484e+03 2.09116797e+03 2.82743100e+03
 3.54672945e+03 4.34745791e+03 5.02654400e+03 1.21500000e+04
 1.38240000e+04 1.56060000e+04 2.40000000e+03 3.21698816e+03
 1.76776695e+03 3.52800000e+03 6.14400000e+03 7.35000000e+03
 2.12176224e+03 2.39313760e+03 1.25663600e+03 8.04247040e+02
 5.83200000e+03 1.60459851e+03 4.02638741e+03 6.82215118e+03
 1.34354395e+04 8.15334068e+03 2.01061760e+04 2.43284730e+04
 3.01718304e+04 5.14718106e+04 1.80123641e+04 2.32351996e+04
 1.77952224e+03 1.38544119e+03 1.20687321e+03 9.95381376e+04
 1.01787516e+05 5.708595

# Total magnetization
Формула: (Ms emu/g * Volume(нм * 10^-21) * density гм/см^3 )/ 9.27 * 10 ** -21 (emu) - спросить как считать объём-то один

In [319]:
# Сначала нужно получить полотность так как у нас её нет
def calculate_density(formula):
    comp = Composition(formula)
    print(comp)
    total_mass = 0
    total_volume = 0
    for element, amount in comp.items():
        total_mass += element.atomic_mass * amount
        print(element, amount, total_mass)
        total_volume += element.atomic_radius ** 3 * amount  # Предполагая, что плотность пропорциональна объему атома
    total_density = total_mass / total_volume
    return total_density

In [320]:
def calculate_total_magn(row):
    # Колонки 'sat_em_g' и 'volume' должны быть volume и saturation magnetization
    ms = row['sat_em_g']
    volume = row['volume']
    density = calculate_density(row['formula_combined'])  # Расчет плотности по формуле
    total_magn = (ms * volume * density) / (9.27 * 10 ** -21)  # Расчет total_magn
    return total_magn

In [321]:
df['total_magn'] = df.apply(calculate_total_magn, axis=1) # Выводит красивые атомные массы 

Zn1 Fe2 Co1 O5
Zn 1.0 65.409
Fe 2.0 177.099
Co 1.0 236.032195
O 5.0 316.029195
Fe2 Co2 O5
Fe 2.0 111.69
Co 2.0 229.55639
O 5.0 309.55339
Mn1 Zn1 Fe4 O8
Mn 1.0 54.938045
Zn 1.0 120.34704500000001
Fe 4.0 343.727045
O 8.0 471.722245
Mn1 Zn1 Fe4 O8
Mn 1.0 54.938045
Zn 1.0 120.34704500000001
Fe 4.0 343.727045
O 8.0 471.722245
Mn1 Zn1 Fe4 O8
Mn 1.0 54.938045
Zn 1.0 120.34704500000001
Fe 4.0 343.727045
O 8.0 471.722245
Mn1 Zn1 Fe4 O8
Mn 1.0 54.938045
Zn 1.0 120.34704500000001
Fe 4.0 343.727045
O 8.0 471.722245
Mn1 Zn1 Fe4 O8
Mn 1.0 54.938045
Zn 1.0 120.34704500000001
Fe 4.0 343.727045
O 8.0 471.722245
La1 Fe1 O3
La 1.0 138.90547
Fe 1.0 194.75047
O 3.0 242.74867
La1 Fe1 O3
La 1.0 138.90547
Fe 1.0 194.75047
O 3.0 242.74867
La1 Fe1 O3
La 1.0 138.90547
Fe 1.0 194.75047
O 3.0 242.74867
La1 Fe1 O3
La 1.0 138.90547
Fe 1.0 194.75047
O 3.0 242.74867
La1 Fe1 O3
La 1.0 138.90547
Fe 1.0 194.75047
O 3.0 242.74867
La1 Fe1 O3
La 1.0 138.90547
Fe 1.0 194.75047
O 3.0 242.74867
La1 Fe1 O3
La 1.0 138.90547
Fe 1

In [322]:
print(df['total_magn'])

0      2.626833e+26
1      3.471966e+26
2      1.339124e+26
3      3.849666e+25
4      3.321658e+26
           ...     
528    1.106397e+27
529    1.123418e+27
530    7.557954e+26
531    7.646898e+26
532    5.792759e+26
Name: total_magn, Length: 533, dtype: float64


# Last

In [323]:
# Выгрузим данные отдельный датасет 
df.to_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\Bias_data_2.csv')